In [11]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:

# Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
)

# Load in the training dataset with augmentation applied
train_generator = train_datagen.flow_from_directory(
    '../binary-brain-classification/training',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

# Only rescale the test set
test_datagen = ImageDataGenerator(
    rescale = 1./255
)

# Load in the test dataset with only rescaling
test_generator = test_datagen.flow_from_directory(
    '../binary-brain-classification/testing',
    target_size = (224,224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4845 images belonging to 2 classes.
Found 392 images belonging to 2 classes.


In [13]:
model = keras.Sequential([

    # Define the input shape
    keras.layers.Input(shape=(224,224,3)),
    
    # CNN layer 1
    keras.layers.Conv2D(16, (3,3), activation=None),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    # Flatten and classify
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation = 'sigmoid')

])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 16)     │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 22, 22, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 22, 22, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 11, 11, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1936)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1936)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         1,937 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,449 (9.57 KB)

 Trainable params: 2,417 (9.44 KB)

 Non-trainable params: 32 (128.00 B)

In [14]:
# Compile the Model
model.compile(
    optimizer = 'adam',    # Popular optimizer
    loss = 'binary_crossentropy',   # Categorical crossentropy for classification tasks
    metrics = (['accuracy'])    # Measure the accuracy
)

In [15]:
# Run with GPU for better performance
with tf.device('/GPU:0'):   
    # Test the model
    model.fit(
        train_generator,    # Traning dataset from our generator
        epochs = 10    # See all the images 10 times
    )

Epoch 1/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.6726 - loss: 0.6798
Epoch 2/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8389 - loss: 0.3998
Epoch 3/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8544 - loss: 0.3569
Epoch 4/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8688 - loss: 0.3323
Epoch 5/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8846 - loss: 0.3106
Epoch 6/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.8841 - loss: 0.3068
Epoch 7/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9088 - loss: 0.2746
Epoch 8/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9077 - loss: 0.2642
Epoch 9/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9173 - loss: 0.2707
Epoch 10/10
152/152 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9161 - loss: 0.2482


In [16]:
model.evaluate(test_generator)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5815 - loss: 1.1696


[1.2433419227600098, 0.5688775777816772]

In [17]:
model.save('./models/binary_model.keras')